In [290]:
import pandas as pd

In [291]:
train_data = pd.read_csv("la_nugs.csv")

In [292]:
test_data = pd.read_csv("test_data.csv")

In [293]:
for i in train_data.columns:
    if i not in ["Team Score","Opp Score","Home/Away","Opponent","Team","Opponent_num"]:
        train_data[i]=train_data[i].shift(1)

In [294]:
test_data.rename({"Tm":"Team Score","Opp":"Opp Score","Team_num":"Team"},inplace=True,axis=1)

In [295]:
for i in test_data.columns:
    if i not in ["Team Score","Opp Score","Home/Away","Opponent","Team","Opponent_num"]:
        test_data[i]=test_data[i].shift(1)

In [296]:
print(train_data.columns)
test_data.drop(['Unnamed: 0','Rk','G','Date'],axis=1,inplace=True)
print(test_data.columns)
print([i for i in train_data.columns if i not in test_data.columns])
print([i for i in train_data.columns if i in test_data.columns])

Index(['Team Score', 'Opp Score', 'Home/Away', 'Opponent', 'Team FG%',
       'Opponent FG%', 'Team 3P%', 'Opponent 3P%', 'Team AST', 'Team TOV',
       'Opponent AST', 'Opponent TOV', 'Team FGA', 'Team FTA', 'Opponent FGA',
       'Opponent FTA', 'Team ORB', 'Opponent ORB', 'ORtg', 'DRtg',
       'Ast_Tov_Team', 'Ast_Tov_Opp', 'Possession', 'Team', 'Opponent_num'],
      dtype='object')
Index(['Home/Away', 'Opponent', 'W/L', 'Team Score', 'Opp Score', 'ORtg',
       'DRtg', 'Pace', 'FTr', '3PAr', 'TS%', 'TRB%', 'AST%', 'STL%', 'BLK%',
       'Unnamed: 18_level_1', 'eFG%', 'TOV%', 'ORB%', 'FT/FGA',
       'Unnamed: 23_level_1', 'def_eFG%', 'def_TOV%', 'def_DRB%', 'def_FT/FGA',
       'Team', 'Opp_num', 'Opponent_num'],
      dtype='object')
['Team FG%', 'Opponent FG%', 'Team 3P%', 'Opponent 3P%', 'Team AST', 'Team TOV', 'Opponent AST', 'Opponent TOV', 'Team FGA', 'Team FTA', 'Opponent FGA', 'Opponent FTA', 'Team ORB', 'Opponent ORB', 'Ast_Tov_Team', 'Ast_Tov_Opp', 'Possession']
['Team 

In [297]:
train_data.fillna(0,inplace=True)
train_x = train_data[["Team","Home/Away","Opponent_num","ORtg","DRtg"]].to_numpy()
train_y = train_data[["Team Score"]].to_numpy()

In [298]:
test_data.fillna(0,inplace=True)
test_x = test_data[["Team","Home/Away","Opponent_num","ORtg","DRtg"]].to_numpy()
test_y = test_data[["Team Score"]].to_numpy()

In [299]:
import numpy as np
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True, normalize=True)

# Train the model
model = model.fit(train_x,train_y)

_sq = model.score(test_x, test_y)

print(_sq)

-0.29315480396607674


In [300]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(train_x, train_y)
knn_sq = knn.score(test_x,test_y)
print(knn_sq)

0.0


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [301]:
# LinearSVC classification (Accuracy of 68.46%)
from sklearn.svm import LinearSVC
lclf = LinearSVC(random_state=0, tol=1e-5, max_iter=1000)
lclf.fit(train_x, train_y)
svc_sq = lclf.score(test_x,test_y)
print(svc_sq)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.07142857142857142


/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [306]:
# SVC prediction model attempt 
from sklearn.svm import SVC
from sklearn import svm
clf = SVC(gamma='scale',probability=True)
clf.fit(train_x, train_y)
print(clf.score(test_x,test_y))

#BaggingSVC Ensemble classification 
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
advclf = BaggingClassifier(base_estimator=SVC(gamma='scale'), n_estimators=10, random_state=0)
advclf.fit(train_x, train_y)
print(advclf.score(test_x,test_y))

# Random Forest Classification model 
from sklearn.ensemble import RandomForestClassifier
rdf = RandomForestClassifier(max_depth=8, random_state=0, n_estimators=300)
rdf.fit(train_x, train_y)
print(rdf.score(test_x,test_y))

# # XGBClassifier
# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score
# xgbModel = XGBClassifier()
# xgbModel.fit(train_x, train_y)
# xgbPredictions = xgbModel.predict(test_x)
# xgbAccuracy = accuracy_score(test_y, xgbPredictions)
# print(xgbAccuracy)

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.07142857142857142


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.0


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.07142857142857142


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

<b>Algorithms to only predict wins</b>

In [317]:
test_data["Win"] = (test_data["Team Score"]>test_data["Opp Score"]).astype("int")
train_data["Win"] = (train_data["Team Score"]>train_data["Opp Score"]).astype("int")

In [318]:
test_data

,Home/Away,Opponent,W/L,Team Score,Opp Score,ORtg,DRtg,Pace,FTr,3PAr,...,FT/FGA,Unnamed: 23_level_1,def_eFG%,def_TOV%,def_DRB%,def_FT/FGA,Team,Opp_num,Opponent_num,Win
0,1,UTA,0,135,125,0.0,0.0,0.0,0.000,0.000,...,0.000,0.0,0.000,0.0,0.0,0.000,18,0.0,21,1
1,1,UTA,W,105,124,130.0,120.4,94.0,0.189,0.432,...,0.158,0.0,0.557,12.3,67.3,0.175,18,0.0,21,0
2,0,UTA,L,87,124,122.7,144.8,85.6,0.284,0.333,...,0.222,0.0,0.632,6.0,71.8,0.161,18,0.0,21,0
3,0,UTA,L,127,129,95.4,135.9,91.2,0.125,0.432,...,0.102,0.0,0.619,9.5,69.0,0.238,18,0.0,21,0
4,1,UTA,L,117,107,139.2,141.4,91.2,0.130,0.440,...,0.120,0.0,0.671,10.1,79.3,0.425,18,0.0,21,1
5,0,UTA,W,119,107,123.8,113.2,94.5,0.188,0.412,...,0.176,0.0,0.603,15.0,89.2,0.167,18,0.0,21,1
6,1,UTA,W,80,78,131.8,118.5,90.3,0.366,0.507,...,0.324,0.0,0.573,10.2,87.8,0.280,18,0.0,21,1
7,0,LAC,W,97,120,91.6,89.3,87.3,0.157,0.373,...,0.120,0.0,0.430,12.3,79.2,0.127,18,0.0,17,0
8,0,LAC,L,110,101,101.4,125.4,95.7,0.325,0.434,...,0.217,0.0,0.631,13.3,75.7,0.167,18,0.0,17,1
9,1,LAC,W,107,113,116.2,106.7,94.7,0.187,0.440,...,0.143,0.0,0.460,14.4,62.0,0.227,18,0.0,17,0


In [319]:
train_data.fillna(0,inplace=True)
train_x = train_data[["Team","Home/Away","Opponent_num","ORtg","DRtg"]].to_numpy()
train_y = train_data[["Win"]].to_numpy()
test_data.fillna(0,inplace=True)
test_x = test_data[["Team","Home/Away","Opponent_num","ORtg","DRtg"]].to_numpy()
test_y = test_data[["Win"]].to_numpy()

In [320]:
import numpy as np
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True, normalize=True)

# Train the model
model = model.fit(train_x,train_y)

_sq = model.score(test_x, test_y)

print(_sq)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(train_x, train_y)
knn_sq = knn.score(test_x,test_y)
print(knn_sq)

# LinearSVC classification (Accuracy of 68.46%)
from sklearn.svm import LinearSVC
lclf = LinearSVC(random_state=0, tol=1e-5, max_iter=1000)
lclf.fit(train_x, train_y)
svc_sq = lclf.score(test_x,test_y)
print(svc_sq)

# SVC prediction model attempt 
from sklearn.svm import SVC
from sklearn import svm
clf = SVC(gamma='scale',probability=True)
clf.fit(train_x, train_y)
print(clf.score(test_x,test_y))

#BaggingSVC Ensemble classification 
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
advclf = BaggingClassifier(base_estimator=SVC(gamma='scale'), n_estimators=10, random_state=0)
advclf.fit(train_x, train_y)
print(advclf.score(test_x,test_y))

# Random Forest Classification model 
from sklearn.ensemble import RandomForestClassifier
rdf = RandomForestClassifier(max_depth=8, random_state=0, n_estimators=300)
rdf.fit(train_x, train_y)
print(rdf.score(test_x,test_y))

# XGBClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgbModel = XGBClassifier()
xgbModel.fit(train_x, train_y)
xgbPredictions = xgbModel.predict(test_x)
xgbAccuracy = accuracy_score(test_y, xgbPredictions)
print(xgbAccuracy)

-0.0029732732694591757
0.42857142857142855


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5
0.42857142857142855


/usr/local/lib/python3.7/site-packages/sklearn/ensemble/_bagging.py:645: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.42857142857142855


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.5714285714285714


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5


In [ ]:
train_data.fillna(0,inplace=True)
train_x["Exponential Rolling ORTg Average"] = train_data["ORtg"].rolling(10,win_type=exponential).mean()
train_x["Exponential Rolling DRTg Average"] = train_data["ORtg"].rolling(10,win_type=exponential).mean()
test_x["Exponential Rolling ORTg Average"] = test_data["ORtg"].rolling(10,win_type=exponential).mean()
test_x["Exponential Rolling DRTg Average"] = test_data["ORtg"].rolling(10,win_type=exponential).mean()
train_x = train_data[["Team","Home/Away","Opponent_num","Exponential Rolling ORTg Average","Exponential Rolling DRTg Average"]].to_numpy()
train_y = train_data[["Win"]].to_numpy()
test_data.fillna(0,inplace=True)
test_x = test_data[["Team","Home/Away","Opponent_num","Exponential Rolling ORTg Average","Exponential Rolling DRTg Average"]].to_numpy()
test_y = test_data[["Win"]].to_numpy()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=True, normalize=True)

# Train the model
model = model.fit(train_x,train_y)

_sq = model.score(test_x, test_y)

print(_sq)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(train_x, train_y)
knn_sq = knn.score(test_x,test_y)
print(knn_sq)

# LinearSVC classification (Accuracy of 68.46%)
from sklearn.svm import LinearSVC
lclf = LinearSVC(random_state=0, tol=1e-5, max_iter=1000)
lclf.fit(train_x, train_y)
svc_sq = lclf.score(test_x,test_y)
print(svc_sq)

# SVC prediction model attempt 
from sklearn.svm import SVC
from sklearn import svm
clf = SVC(gamma='scale',probability=True)
clf.fit(train_x, train_y)
print(clf.score(test_x,test_y))

#BaggingSVC Ensemble classification 
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
advclf = BaggingClassifier(base_estimator=SVC(gamma='scale'), n_estimators=10, random_state=0)
advclf.fit(train_x, train_y)
print(advclf.score(test_x,test_y))

# Random Forest Classification model 
from sklearn.ensemble import RandomForestClassifier
rdf = RandomForestClassifier(max_depth=8, random_state=0, n_estimators=300)
rdf.fit(train_x, train_y)
print(rdf.score(test_x,test_y))

# XGBClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgbModel = XGBClassifier()
xgbModel.fit(train_x, train_y)
xgbPredictions = xgbModel.predict(test_x)
xgbAccuracy = accuracy_score(test_y, xgbPredictions)
print(xgbAccuracy)